## Parameters

In [ ]:
# "convnext" "densenet" "efficientnet" "efficientnet_v2""inception_resnet_v2""inception_v3""mobilenet_v3" "resnet50" "resnet_v2" "vgg16" "vgg19" "xception"
MODEL_ARCHITECTURE = "mobilenet_v3"
EPOCHS = 10
TRANFERT_LEARNING =True
#==============================#

DATASET = "pathmnist_128"
INPUT_SHAPE = (128, 128, 3)
CLASSES = 9
#==============================#
param_grid = {
    "learning_rate": [1e-4, 1e-3],
    "batch_size": [32, 64, 128],
    "optimizer": ["adam", "sgd"],
    "dropout": [0.2, 0.3, 0.4, 0.5],
    "dense_units": [64, 128, 256]
}
#==============================#
base_path = '/content/drive/MyDrive/ColabNotebooks/phd-thesis'

## Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import tensorflow
import pandas as pd
from datetime import datetime

from sklearn.model_selection import ParameterGrid
from tensorflow.keras.models import load_model
sys.path.append(f'{base_path}/lib')

#importing helpers
from import_pathmnist_dataset import PathMNISTLoader
from create_model import create_model
from import_callbacks import get_callbacks
from helper_saving import save_training

experiment_info = {
    "model" : MODEL_ARCHITECTURE,
    "dataset" : DATASET,
    "mode" : "Local",
    "run_time" : datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Dataset

In [3]:
loader = PathMNISTLoader(filename= DATASET, num_classes= CLASSES, batch_size= 64)
train_ds, val_ds, test_ds = loader.get_datasets()

Download complete.
Loaded data keys: ['train_images', 'train_labels', 'val_images', 'val_labels', 'test_images', 'test_labels']
Train: (89996, 128, 128, 3) (89996, 1)
Validation: (10004, 128, 128, 3) (10004, 1)
Test: (7180, 128, 128, 3) (7180, 1)


## Train and Save Results

In [ ]:
# Initialize an empty list to collect results
results = []

for i,config in enumerate(ParameterGrid(param_grid)):
  model = create_model(
    architecture=MODEL_ARCHITECTURE,
    input_shape=INPUT_SHAPE,
    num_classes=CLASSES,
    dropout_rate=config['dropout'],
    learning_rate=config['learning_rate'],
    optimizer=config['optimizer'],
    dense_units=config['dense_units'],
    freeze_base=TRANFERT_LEARNING
    )
  # Get callbacks
  callbacks = get_callbacks(
      model_name = f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}',
      checkpoint_dir = os.path.join(base_path, MODEL_ARCHITECTURE,'models'),
      )

  # Train the model
  history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=20,
      callbacks=callbacks,
      verbose=2,
  )

  SAVE_PATH=os.path.join(base_path, MODEL_ARCHITECTURE,'results', f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}.xlsx')
  save_training(save_path=SAVE_PATH,model=model,history = history,hyperparams=config,experiment_info=experiment_info)

  model = load_model(os.path.join(base_path, MODEL_ARCHITECTURE,'models',f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}.keras'))
  # Evaluate on the test set
  test_loss, test_accuracy = model.evaluate(test_ds)

  # Get best validation accuracy & loss
  best_val_acc = max(history.history["val_accuracy"])
  best_val_loss = min(history.history["val_loss"])

  # Log all info into a result dictionary
  result_record = {
      "model": MODEL_ARCHITECTURE,
      "run_id": i,
      "test_accuracy": test_accuracy,
      "test_loss": test_loss,
      "best_val_accuracy": best_val_acc,
      "best_val_loss": best_val_loss,
      **config  # unpack all hyperparameters
  }
  results.append(result_record)

results_df = pd.DataFrame(results)
results_df.to_excel(os.path.join(base_path, MODEL_ARCHITECTURE,'results', f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}.xlsx'), index=False)
